In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

max_features = 10000
max_len = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)


In [2]:

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_train[0])

(25000, 20) (25000,)
(25000, 20) (25000,)
[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
   15   16 5345   19  178   32]


In [10]:
word_index = imdb.get_word_index()

index_to_word = {index + 3: word for word, index in word_index.items()}
index_to_word[0] = "<PAD>"
index_to_word[1] = "<START>"
index_to_word[2] = "<UNK>"
index_to_word[3] = "<UNUSED>"

In [5]:
def decode_review(encoded_review):
    return ' '.join([index_to_word.get(i, '?') for i in encoded_review])

decoded_review = decode_review(x_train[0])
print(decode_review)

<function decode_review at 0x3223d7240>


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
print(x_train.shape)
model = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 inputlength를 지정합니다.
model.add(Embedding (10000, 8, input_length=max_len))
# 임베딩 텐서를 (samples, maxlen * 8) 크기의 텐서로 펼칩니다.
model.add(Flatten())
# 분류기를 추가합니다.
model.add(Dense(1, activation='sigmoid')) 
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# model.summary()
history = model. fit(x_train, y_train,
                epochs=10, 
                batch_size=32, 
                validation_split=0.2)

(25000, 20)
Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 469us/step - acc: 0.5632 - loss: 0.6836 - val_acc: 0.6974 - val_loss: 0.6080
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - acc: 0.7419 - loss: 0.5606 - val_acc: 0.7304 - val_loss: 0.5238
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - acc: 0.7853 - loss: 0.4702 - val_acc: 0.7450 - val_loss: 0.5011
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - acc: 0.8043 - loss: 0.4320 - val_acc: 0.7468 - val_loss: 0.4982
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 381us/step - acc: 0.8236 - loss: 0.4000 - val_acc: 0.7574 - val_loss: 0.4975
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - acc: 0.8307 - loss: 0.3787 - val_acc: 0.7592 - val_loss: 0.4994
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - acc: 0.8376 - loss: 0.3654 - val_acc: 0.7562 - val_loss: 0.5041
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - acc: 0.8533 - loss: 0.3466 - val_acc: 0.7532 - val_loss: 0.5108
Epoch 9/10
625/625 ━━━━━━━━━

In [12]:
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (32, 20, 8)            │        80,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (32, 160)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │           161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,324 (626.27 KB)

 Trainable params: 80,161 (313.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 80,163 (313.14 KB)

In [15]:
# 새로운 리뷰를 사용한 검증
new_reviews = [
    "It was a boring movie. I don't want to recommend it.",
    "The movie was extremely disappointing."
]

new_sequences = []
for  review in new_reviews:
    sequence = [word_index.get(word.lower(), 2) for word in review.split()]
    new_sequences.append(sequence)

padded_new_sequences = preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_len)

predictions = model.predict(padded_new_sequences)

for review, prediction in zip(new_reviews, predictions):
    print(f"리뷰: {review} -> 예측: {'긍정' if prediction > 0.5 else '부정'} (확률:{prediction[0]:.4f})")

2024-07-31 10:26:19.646330: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: indices[2,6] = 18581 is not in [0, 10000)
	 [[{{node sequential_3_1/embedding_2_1/GatherV2}}]]


InvalidArgumentError: Graph execution error:

Detected at node sequential_3_1/embedding_2_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/opt/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/opt/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/opt/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/opt/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/opt/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/kernelbase.py", line 534, in

In [19]:
# 새로운 리뷰를 사용한 검증
new_reviews = [
    "I absolutely loved this movie. The storyline was gripping and the characters were very well developed. The performances were outstanding and the direction was top-notch. I would highly recommend this movie to anyone.",
    "This film was an incredible journey. The visuals were stunning and the soundtrack was perfect. The plot was engaging from start to finish and I felt a deep connection to the characters. A must-watch!",
    "The movie was extremely disappointing. The plot was weak and the characters were not believable. The acting was subpar and the film dragged on far too long. I wouldn't recommend this movie.",
    "I found the film to be quite boring. The storyline was predictable and there was no character development. The acting was mediocre at best and the whole movie felt like a waste of time. Definitely not worth watching."
]

new_sequences = []
for  review in new_reviews:
    sequence = [word_index.get(word.lower(), 2) for word in review.split()]
    sequence = [index if index < max_features else 2 for index in sequence]
    new_sequences.append(sequence)

padded_new_sequences = preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_len)

predictions = model.predict(padded_new_sequences)

for review, prediction in zip(new_reviews, predictions):
    print(f"리뷰: {review} -> 예측: {'긍정' if prediction > 0.5 else '부정'} (확률:{prediction[0]:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
리뷰: I absolutely loved this movie. The storyline was gripping and the characters were very well developed. The performances were outstanding and the direction was top-notch. I would highly recommend this movie to anyone. -> 예측: 부정 (확률:0.1635)
리뷰: This film was an incredible journey. The visuals were stunning and the soundtrack was perfect. The plot was engaging from start to finish and I felt a deep connection to the characters. A must-watch! -> 예측: 긍정 (확률:0.7140)
리뷰: The movie was extremely disappointing. The plot was weak and the characters were not believable. The acting was subpar and the film dragged on far too long. I wouldn't recommend this movie. -> 예측: 부정 (확률:0.0546)
리뷰: I found the film to be quite boring. The storyline was predictable and there was no character development. The acting was mediocre at best and the whole movie felt like a waste of time. Definitely not worth watching. -> 예측: 긍정 (확률:0.9908)


imdb 다운로드